In [1]:
import numpy as np 
import pandas as pd 
from statsmodels import api as sm
import pylab as py
import matplotlib.pyplot as plt
import matplotlib.dates as dates
from datetime import datetime
import matplotlib.ticker as ticker
import matplotlib.cm as cm
import matplotlib as mpl
from matplotlib.gridspec import GridSpec
import seaborn as sns
import squarify
from scipy.stats import kstest,norm
from scipy.stats import norm
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from xgboost import plot_importance
from sklearn.utils import resample
from sklearn import metrics
from scipy.stats import chi2_contingency

In [147]:
df = pd.read_csv(r'C:\Users\ranu1\Downloads\E-comercedataset\2019-Nov.csv')

In [148]:
df.head(10)

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-11-01 00:00:00 UTC,view,1003461,2053013555631882655,electronics.smartphone,xiaomi,489.07,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33
1,2019-11-01 00:00:00 UTC,view,5000088,2053013566100866035,appliances.sewing_machine,janome,293.65,530496790,8e5f4f83-366c-4f70-860e-ca7417414283
2,2019-11-01 00:00:01 UTC,view,17302664,2053013553853497655,NaN,creed,28.31,561587266,755422e7-9040-477b-9bd2-6a6e8fd97387
3,2019-11-01 00:00:01 UTC,view,3601530,2053013563810775923,appliances.kitchen.washer,lg,712.87,518085591,3bfb58cd-7892-48cc-8020-2f17e6de6e7f
4,2019-11-01 00:00:01 UTC,view,1004775,2053013555631882655,electronics.smartphone,xiaomi,183.27,558856683,313628f1-68b8-460d-84f6-cec7a8796ef2
5,2019-11-01 00:00:01 UTC,view,1306894,2053013558920217191,computers.notebook,hp,360.09,520772685,816a59f3-f5ae-4ccd-9b23-82aa8c23d33c
6,2019-11-01 00:00:01 UTC,view,1306421,2053013558920217191,computers.notebook,hp,514.56,514028527,df8184cc-3694-4549-8c8c-6b5171877376
7,2019-11-01 00:00:02 UTC,view,15900065,2053013558190408249,NaN,rondell,30.86,518574284,5e6ef132-4d7c-4730-8c7f-85aa4082588f
8,2019-11-01 00:00:02 UTC,view,12708937,2053013553559896355,NaN,michelin,72.72,532364121,0a899268-31eb-46de-898d-09b2da950b24
9,2019-11-01 00:00:02 UTC,view,1004258,2053013555631882655,electronics.smartphone,apple,732.07,532647354,d2d3d2c6-631d-489e-9fb5-06f340b85be0


In [149]:
df.count()

event_time       67501979
event_type       67501979
product_id       67501979
category_id      67501979
category_code    45603808
brand            58277901
price            67501979
user_id          67501979
user_session     67501969
dtype: int64

In [150]:
df.isnull().sum()

event_time              0
event_type              0
product_id              0
category_id             0
category_code    21898171
brand             9224078
price                   0
user_id                 0
user_session           10
dtype: int64

In [151]:
print("category_code ",df['category_code'].isnull().sum())
print("brand ",df['brand'].isnull().sum())
print("Both ",(df['category_code'].isnull() & df['brand'].isnull()).sum())

category_code  21898171
brand  9224078
Both  5709840


In [152]:
df = df.dropna()

In [153]:
df.shape

(42089564, 9)

In [154]:
df.isnull().sum()

event_time       0
event_type       0
product_id       0
category_id      0
category_code    0
brand            0
price            0
user_id          0
user_session     0
dtype: int64

In [155]:
num_users = df['user_id'].nunique()
num_products = df['product_id'].nunique()
num_categories = df['category_id'].nunique()
num_taxonomies = df['category_code'].nunique()
num_brands = df['brand'].nunique()
print('Total number of users: ',num_users)
print('Total number of products: ', num_products)
print('Total number of categories: ', num_categories)
print('Total number of taxonomies: ', num_taxonomies)
print('Total number of brands: ', num_brands)

Total number of users:  2862935
Total number of products:  69773
Total number of categories:  270
Total number of taxonomies:  129
Total number of brands:  1987


In [156]:
df = df[['category_code','user_id']]

In [157]:
df.shape

(42089564, 2)

In [158]:
df

,category_code,user_id
0,electronics.smartphone,520088904
1,appliances.sewing_machine,530496790
3,appliances.kitchen.washer,518085591
4,electronics.smartphone,558856683
5,computers.notebook,520772685
...,...,...
67501972,construction.tools.drill,562661595
67501975,apparel.shoes,545223467
67501976,electronics.smartphone,557794415
67501977,appliances.kitchen.refrigerators,531607492


In [17]:
unique_items_per_user = df.groupby('user_id')['category_code'].apply(lambda x: x.unique())
print(unique_items_per_user)

user_id
29515875         [furniture.bedroom.bed, electronics.video.tv]
31198833     [electronics.smartphone, appliances.kitchen.gr...
34916060                            [appliances.kitchen.grill]
41798457                              [electronics.smartphone]
62336140                                       [apparel.shoes]
                                   ...                        
579969525                             [electronics.smartphone]
579969533                               [electronics.video.tv]
579969570                             [electronics.smartphone]
579969767                        [electronics.audio.headphone]
579969851                             [electronics.smartphone]
Name: category_code, Length: 2862935, dtype: object


In [21]:
type(unique_items_per_user)

pandas.core.series.Series

In [22]:
unique_items_df = unique_items_per_user.reset_index()
print(unique_items_df)

           user_id                                      category_code
0         29515875      [furniture.bedroom.bed, electronics.video.tv]
1         31198833  [electronics.smartphone, appliances.kitchen.gr...
2         34916060                         [appliances.kitchen.grill]
3         41798457                           [electronics.smartphone]
4         62336140                                    [apparel.shoes]
...            ...                                                ...
2862930  579969525                           [electronics.smartphone]
2862931  579969533                             [electronics.video.tv]
2862932  579969570                           [electronics.smartphone]
2862933  579969767                      [electronics.audio.headphone]
2862934  579969851                           [electronics.smartphone]

[2862935 rows x 2 columns]


In [53]:
unique_items_df['category_code']

(2862935,)

In [23]:
unique_items_df.head()

,user_id,category_code
0,29515875,"[furniture.bedroom.bed, electronics.video.tv]"
1,31198833,"[electronics.smartphone, appliances.kitchen.gr..."
2,34916060,[appliances.kitchen.grill]
3,41798457,[electronics.smartphone]
4,62336140,[apparel.shoes]


In [28]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

# Example DataFrame
data = unique_items_df

df = pd.DataFrame(data)

# Initialize MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# Apply MultiLabelBinarizer to the category_code column
category_encoded = mlb.fit_transform(df['category_code'])

# Create a new DataFrame with the encoded category columns
df_encoded = pd.DataFrame(category_encoded, columns=mlb.classes_)

# Concatenate the original DataFrame with the encoded categories
df_final = pd.concat([df['user_id'], df_encoded], axis=1)

# Display the final DataFrame
print(df_final)


           user_id  accessories.bag  accessories.umbrella  accessories.wallet  \
0         29515875                0                     0                   0   
1         31198833                0                     0                   0   
2         34916060                0                     0                   0   
3         41798457                0                     0                   0   
4         62336140                0                     0                   0   
...            ...              ...                   ...                 ...   
2862930  579969525                0                     0                   0   
2862931  579969533                0                     0                   0   
2862932  579969570                0                     0                   0   
2862933  579969767                0                     0                   0   
2862934  579969851                0                     0                   0   

         apparel.belt  appa

In [30]:
type(df_final)

pandas.core.frame.DataFrame

In [31]:
df_final.head()

,user_id,accessories.bag,accessories.umbrella,accessories.wallet,apparel.belt,apparel.costume,apparel.dress,apparel.glove,apparel.jacket,apparel.jeans,...,kids.skates,kids.swing,kids.toys,medicine.tools.tonometer,sport.bicycle,sport.ski,sport.snowboard,sport.tennis,sport.trainer,stationery.cartrige
0,29515875,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,31198833,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,34916060,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,41798457,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,62336140,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
df_final.shape  #130 unique category code we have in our code

(2862935, 130)

In [63]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# Example DataFrame
data = unique_items_df

df = pd.DataFrame(data)

# Convert the list of categories into a one-hot encoded format
te = TransactionEncoder()
te_ary = te.fit(df['category_code']).transform(df['category_code'])
df_onehot = pd.DataFrame(te_ary, columns=te.columns_)

# Apply the Apriori algorithm with a minimum support threshold (e.g., 0.4)
frequent_itemsets = apriori(df_onehot, min_support=0.004, use_colnames=True)

# Generate association rules with a minimum lift threshold (e.g., 1.0)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

# Display the results
print("Frequent Itemsets:")
print(frequent_itemsets)
print("\nAssociation Rules:")
print(rules)


Frequent Itemsets:
      support                                           itemsets
0    0.019332                                  (accessories.bag)
1    0.004330                               (accessories.wallet)
2    0.013990                                  (apparel.costume)
3    0.010183                                    (apparel.dress)
4    0.004804                                   (apparel.jumper)
..        ...                                                ...
175  0.016573  (electronics.audio.headphone, electronics.cloc...
176  0.004153  (electronics.audio.headphone, electronics.smar...
177  0.008303  (electronics.audio.headphone, electronics.vide...
178  0.004013  (electronics.telephone, electronics.clocks, el...
179  0.007519  (electronics.video.tv, electronics.smartphone,...

[180 rows x 2 columns]

Association Rules:
                                      antecedents  \
0                                 (apparel.shoes)   
1                               (accessories.bag)  

In [34]:
pip install mlxtend

   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
    --------------------------------------- 0.0/1.4 MB 1.3 MB/s eta 0:00:02
   ---- ----------------------------------- 0.2/1.4 MB 2.6 MB/s eta 0:00:01
   ------------ --------------------------- 0.5/1.4 MB 4.0 MB/s eta 0:00:01
   --------------------- ------------------ 0.8/1.4 MB 5.0 MB/s eta 0:00:01
   ---------------------------- ----------- 1.0/1.4 MB 5.1 MB/s eta 0:00:01
   ------------------------------------ --- 1.3/1.4 MB 5.2 MB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 4.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


0.007692307692307693

In [56]:
frequent_itemsets.tail()

,support,itemsets
129,0.007669,"(electronics.clocks, electronics.smartphone, c..."
130,0.006109,"(electronics.video.tv, electronics.smartphone,..."
131,0.016573,"(electronics.audio.headphone, electronics.cloc..."
132,0.008303,"(electronics.audio.headphone, electronics.vide..."
133,0.007519,"(electronics.video.tv, electronics.smartphone,..."


In [64]:
frequent_itemsets.shape

(180, 2)

In [59]:
rules.tail(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
122,"(electronics.video.tv, electronics.smartphone)",(electronics.audio.headphone),0.039573,0.110155,0.008303,0.209824,1.904798,0.003944,1.126134,0.494582
123,(electronics.audio.headphone),"(electronics.video.tv, electronics.smartphone)",0.110155,0.039573,0.008303,0.075379,1.904798,0.003944,1.038725,0.533812
124,(electronics.video.tv),"(electronics.audio.headphone, electronics.smar...",0.091013,0.056853,0.008303,0.091233,1.604710,0.003129,1.037831,0.414565
125,(electronics.smartphone),"(electronics.audio.headphone, electronics.vide...",0.550223,0.012193,0.008303,0.015091,1.237626,0.001594,1.002942,0.426882
126,"(electronics.video.tv, electronics.smartphone)",(electronics.clocks),0.039573,0.093068,0.007519,0.190000,2.041510,0.003836,1.119668,0.531187
127,"(electronics.video.tv, electronics.clocks)",(electronics.smartphone),0.010359,0.550223,0.007519,0.725832,1.319159,0.001819,1.640516,0.244474
128,"(electronics.clocks, electronics.smartphone)",(electronics.video.tv),0.050175,0.091013,0.007519,0.149852,1.646497,0.002952,1.069211,0.413392
129,(electronics.video.tv),"(electronics.clocks, electronics.smartphone)",0.091013,0.050175,0.007519,0.082613,1.646497,0.002952,1.035359,0.431964
130,(electronics.smartphone),"(electronics.video.tv, electronics.clocks)",0.550223,0.010359,0.007519,0.013665,1.319159,0.001819,1.003352,0.537915
131,(electronics.clocks),"(electronics.video.tv, electronics.smartphone)",0.093068,0.039573,0.007519,0.080789,2.041510,0.003836,1.044838,0.562519


In [65]:
rules.shape

(232, 10)

In [71]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# Example DataFrame
data = unique_items_df

df = pd.DataFrame(data)

# Convert the list of categories into a one-hot encoded format
te = TransactionEncoder()
te_ary = te.fit(df['category_code']).transform(df['category_code'])
df_onehot = pd.DataFrame(te_ary, columns=te.columns_)

# Apply the Apriori algorithm with a lower min_support
frequent_itemsets = apriori(df_onehot, min_support=0.0045, use_colnames=True)

# Generate association rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

# Filter rules to ensure each item appears in at least one of the antecedent or consequent
item_list = te.columns_
rules_with_items = pd.DataFrame()

for item in item_list:
    item_rules = rules[(rules['antecedents'].apply(lambda x: item in x)) | (rules['consequents'].apply(lambda x: item in x))]
    rules_with_items = pd.concat([rules_with_items, item_rules])

# Remove duplicates if necessary
rules_with_items = rules_with_items.drop_duplicates()

# Export the filtered rules to a CSV file
rules_with_items.to_csv('filtered_association_rules.csv', index=False)

print(rules_with_items)


                  antecedents                                     consequents  \
0             (apparel.shoes)                               (accessories.bag)   
1           (accessories.bag)                                 (apparel.shoes)   
2           (accessories.bag)                            (computers.notebook)   
3        (computers.notebook)                               (accessories.bag)   
4        (electronics.clocks)                               (accessories.bag)   
..                        ...                                             ...   
185      (electronics.clocks)  (electronics.video.tv, electronics.smartphone)   
78   (electronics.smartphone)                            (electronics.tablet)   
79       (electronics.tablet)                        (electronics.smartphone)   
80    (electronics.telephone)                        (electronics.smartphone)   
81   (electronics.smartphone)                         (electronics.telephone)   

     antecedent support  co

In [66]:
rules.to_csv('output1.csv', index=False)


In [263]:
rules_with_items.to_csv('output1.csv', index=False)

In [73]:
type(rules_with_items)

pandas.core.frame.DataFrame

In [133]:
ap = input("Enter category")

Enter category electronics.tablet


In [134]:
filtered_rules = rules_with_items[rules_with_items['antecedents'].apply(lambda x: ap in x)]

In [135]:
a = str(filtered_rules.iloc[0][1])

C:\Users\ranu1\AppData\Local\Temp\ipykernel_11824\1341211584.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  a = str(filtered_rules.iloc[0][1])


In [136]:
print(a)

frozenset({'computers.notebook'})


In [137]:
flag = False
start = 0
end = 0
for i in range(len(a)):
    if a[i] == "'" and flag == False:
        start = i
        flag = True
    if flag == True and a[i] == "'":
        end = i
ans = a[start+1:end]

In [138]:
ans

'computers.notebook'

In [269]:
rules.to_csv('terafile.csv', index=False)

In [159]:
df

,category_code,user_id
0,electronics.smartphone,520088904
1,appliances.sewing_machine,530496790
3,appliances.kitchen.washer,518085591
4,electronics.smartphone,558856683
5,computers.notebook,520772685
...,...,...
67501972,construction.tools.drill,562661595
67501975,apparel.shoes,545223467
67501976,electronics.smartphone,557794415
67501977,appliances.kitchen.refrigerators,531607492


In [191]:
df = df.drop_duplicates(subset='user_id')

In [261]:
df.shape

(2862935, 2)

In [255]:
user = int(input("Enter userid"))

Enter userid 520772685


In [256]:
temp = df[df['user_id']==user]

In [257]:
ap = temp.iloc[0][0]

C:\Users\ranu1\AppData\Local\Temp\ipykernel_11824\1377005968.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ap = temp.iloc[0][0]


In [258]:
ap

'computers.notebook'

In [259]:
filtered_rules = rules_with_items[rules_with_items['antecedents'].apply(lambda x: ap in x)]

In [260]:
if filtered_rules.shape[0] == 0:
    print("No recommendation")
else:
    a = str(filtered_rules.iloc[0][1])
    print(a)

frozenset({'accessories.bag'})


C:\Users\ranu1\AppData\Local\Temp\ipykernel_11824\499648257.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  a = str(filtered_rules.iloc[0][1])


In [268]:
df.to_csv("merafile.csv",index=False)